In [1]:
# Install required libraries: Streamlit, Pillow for image handling
!pip install streamlit pillow -q

# Clone the YOLOv5 repository (needed to load the model using torch.hub)
!git clone https://github.com/ultralytics/yolov5  # The model will be loaded from this cloned repo

# Install PyTorch dependencies from the cloned repo
!pip install -r yolov5/requirements.txt -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 52.4 MB/s eta 0:00:00
Cloning into 'yolov5'...
remote: Enumerating objects: 17760, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 17760 (delta 74), reused 39 (delta 39), pack-reused 17639 (from 4)
Receiving objects: 100% (17760/17760), 17.12 MiB | 24.34 MiB/s, done.
Resolving deltas: 100% (12061/12061), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.2 MB/s eta 0:00:00


In [2]:
%%writefile app.py

import streamlit as st
import torch
from PIL import Image
import numpy as np
import os

# --- Model and Detection Setup ---
# Colab environment often requires the model to be loaded without caching
def load_model():
    try:
        # We explicitly point to the cloned repo
        model = torch.hub.load('yolov5', 'yolov5s', source='local', pretrained=True)
        model.classes = [0] # Only detect 'person'
        return model
    except Exception as e:
        st.error(f"Error loading YOLOv5 model: {e}")
        st.stop()

# Load the model outside the main function so it's loaded only once
MODEL = load_model()

# --- Waiting Time Logic ---
def estimate_wait_time(person_count):
    """
    Converts the number of detected people into an estimated wait time
    with highly granular categories for better demo presentation.
    """
    if person_count == 0:
        return "0 minutes (Walk-in available)"
    elif person_count == 1:
        return "1 - 3 minutes"
    elif person_count == 2:
        return "3 - 6 minutes"
    elif person_count <= 4:
        return "6 - 10 minutes"
    elif person_count <= 6:
        return "10 - 15 minutes"
    elif person_count <= 8:
        return "15 - 20 minutes"
    elif person_count <= 12:
        return "20 - 30 minutes"
    elif person_count <= 18:
        return "30 - 45 minutes"
    elif person_count <= 25:
        return "45 - 60 minutes (1 hour)"
    elif person_count <= 35:
        return "60 - 90 minutes (1 to 1.5 hours)"
    elif person_count <= 50:
        return "90 - 120 minutes (1.5 to 2 hours)"
    else:
        return "Over 2 hours (Maximum Wait/Stop Seating)"
# --- Streamlit App Layout ---
def main():
    st.set_page_config(
        page_title="Restaurant Wait Time Predictor",
        page_icon="🍽️",
        layout="wide"
    )

    st.title("🍽️ Live Restaurant Wait Time Estimator")
    st.markdown("Upload a CCTV image of the restaurant waiting area to estimate the current wait time.")

    uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

    if uploaded_file is not None:
        try:
            # Open the image
            image = Image.open(uploaded_file).convert("RGB")
            st.subheader("Original Image")
            st.image(image, caption='Image uploaded successfully.', use_column_width=True)

            with st.spinner('Analyzing image and counting people...'):
                # Perform YOLOv5 inference
                results = MODEL(image)

                # Render bounding boxes onto the image
                results.render()
                detected_image_array = results.ims[0]
                detected_image = Image.fromarray(detected_image_array)

                # Get the count of detected persons
                person_detections = results.pandas().xyxy[0]
                person_count = len(person_detections)

            # --- Results and Wait Time Display ---
            st.divider()

            col1, col2 = st.columns([1, 1])

            with col1:
                st.subheader("🔍 Detection Result")
                st.image(detected_image, caption=f'Detected {person_count} people.', use_column_width=True)

            with col2:
                st.subheader("⏱️ Estimated Wait Time")
                wait_time = estimate_wait_time(person_count)

                st.metric(label="Total People Detected", value=person_count)
                st.success(f"**Estimated Waiting Time:** {wait_time}")

                st.info("""
                **Project Note:** This waiting time is based on a heuristic function (simulated ML model output).
                The full project includes a dedicated model trained on 30 days of image-to-wait-time data.
                """)


        except Exception as e:
            st.error(f"An error occurred during processing: {e}")

if __name__ == '__main__':
    main()

Writing app.py


In [4]:
# Install ngrok
!pip install pyngrok -q

In [5]:
from pyngrok import ngrok

# !!! IMPORTANT: REPLACE "YOUR_AUTHTOKEN_HERE" with your actual ngrok token !!!
ngrok.set_auth_token("36vTC8hyivHMeCQ2qlXNcEjUMn1_3eFre17kZJEugHpbg7mPu")
print("ngrok Authentication Token has been set.")

ngrok Authentication Token has been set.


In [6]:
# Run Streamlit in the background and use ngrok to expose it
# Port 8501 is the default Streamlit port
from pyngrok import ngrok
import subprocess

# Kill any previous ngrok tunnels
ngrok.kill()

# Start streamlit in the background
subprocess.Popen(['streamlit', 'run', 'app.py'])

# Set up ngrok tunnel to the streamlit port (8501)
# The output will show a link.
tunnel = ngrok.connect(8501)
print(f"Public URL for the Streamlit App: {tunnel.public_url}")

Public URL for the Streamlit App: https://eliminatory-alcoholically-genna.ngrok-free.dev
